In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyfolio as pf
import pandas_ta as ta
import seaborn as sns

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier


import empyrical as ep
import seaborn as sns
import datetime
import plotly.express as px

import plotly.graph_objects as go

In [ ]:
start = "2019-1-1"
df = yf.download(tickers="AAPL", start=start)

df = df.reset_index()

max_drawdown = 30

x = len(df)

trend = ''

Peak = -np.inf
date_Peak = 0
Trough = np.inf
date_Trough = 0

ddd = np.empty((0, 3), np.datetime64)

for i in range(0, x):
    up = 0
    dn = 0
    
    if trend == '' or trend == 'bull':
        if df.loc[i, 'Close'] >= Peak:
            Peak = df.loc[i, 'Close']
            date_Peak = df.loc[i, 'Date']
            
    if trend == '' or trend == 'bear':
        if df.loc[i, 'Close'] <= Trough:
            Trough = df.loc[i, 'Close']
            date_Trough = df.loc[i, 'Date']
            
    if Peak != -np.inf:
        dn = (Peak - df.loc[df.index[i], 'Close']) / (Peak / 100.0)
        
    if Trough != np.inf:
        up = (df.loc[df.index[i], 'Close'] - Trough) / (Trough / 100.0)

    if up >= max_drawdown:
        trend = 'bull'
        ddd = np.append(ddd, np.array([[date_Trough, df.loc[i, 'Date'], 1]]), axis=0)
        Trough = np.inf
        Peak = df.loc[df.index[i], 'Close']
        date_Peak = df.loc[i, 'Date']
        
    if dn >= max_drawdown:
        trend = 'bear'
        ddd = np.append(ddd, np.array([[date_Peak, df.loc[i, 'Date'], 2]]), axis=0)
        Peak = -np.inf
        Trough = df.loc[df.index[i], 'Close']
        date_Trough = df.loc[i, 'Date']


df = df.set_index('Date')
up_trend_s = ddd[ddd[:, 2] == 1, 0]
up_trend_f = ddd[ddd[:, 2] == 1, 1]
dn_trend_s = ddd[ddd[:, 2] == 2, 0]
dn_trend_f = ddd[ddd[:, 2] == 2, 1]

if ddd[len(ddd) - 1, 2] == 1:
    # if the trend is growing, then we are looking for a local maximum
    
    ind = up_trend_f[len(up_trend_f) - 1] #get the index of the beginning of the bullish trend
    imax = df.loc[ind:, 'Close'].idxmax() #local maximum
    
    if df.loc[imax, 'Close'] > df.loc[ind, 'Close']: #if the high is greater than the price of the beginning of the bullish trend, then add a peak
        dn_trend_s = np.append(dn_trend_s, imax)
        
else:
    ind = dn_trend_f[len(dn_trend_f) - 1]#bear market
    imin = df.loc[ind:, 'Close'].idxmin()
    if df.loc[imin, 'Close'] < df.loc[ind, 'Close']:
        up_trend_s = np.append(up_trend_s, imin)



fig, ax = plt.subplots()
ax.plot(df.index, df['Close'])
ax.plot(up_trend_s, df.loc[up_trend_s, 'Close'], 'x', color='lime')
ax.plot(up_trend_f, df.loc[up_trend_f, 'Close'], 'o', color='lime', markersize=4)
ax.plot(dn_trend_s, df.loc[dn_trend_s, 'Close'], 'x', color='red')
ax.plot(dn_trend_f, df.loc[dn_trend_f, 'Close'], 'o', color='red', markersize=4)
fig.autofmt_xdate()
plt.show()

In [ ]:
dft = yf.Ticker("^IXIC") ## NASDAQ Composite (0.07133)
# dft = yf.Ticker("^GSPC") ## S&P 500
# dft = yf.Ticker("TDEX.BK") ## TDEX
# dft = yf.Ticker("^DJI") ## Dow Jones Industrial Average 
# dft = yf.Ticker("AWC.BK") ## Asset World Corp Public Company Limited


tickers = {
    "^GSPC": "S&P 500",
    "^DJI": "Dow Jones Industrial Average",
    "^IXIC": "NASDAQ Composite",
    "^NYA": "NYSE COMPOSITE (DJ)",
    "^XAX": "NYSE AMEX COMPOSITE INDEX",
    "^BUK100P": "Cboe UK 100",
    "^RUT": "Russell 2000",
    "^VIX": "CBOE Volatility Index",
    "^FTSE": "FTSE 100",
    "^GDAXI": "DAX PERFORMANCE-INDEX",
    "^FCHI": "CAC 40",
    "^STOXX50E": "ESTX 50 PR.EUR",
    "^N100": "Euronext 100 Index",
    "^BFX": "BEL 20",
    "IMOEX.ME": "MOEX Russia Index",
    "^N225": "Nikkei 225",
    "^HSI": "HANG SENG INDEX",
    "000001.SS": "SSE Composite Index",
    "399001.SZ": "Shenzhen Index",
    "^STI": "STI Index",
    "^AXJO": "S&P/ASX 200",
    "^AORD": "ALL ORDINARIES",
    "^BSESN": "S&P BSE SENSEX",
    "^JKSE": "IDX COMPOSITE",
    "^KLSE": "FTSE Bursa Malaysia KLCI",
    "^NZ50": "S&P/NZX 50 INDEX GROSS (GROSS)",
    "^KS11": "KOSPI Composite Index",
    "^TWII": "TSEC weighted index",
    "^GSPTSE": "S&P/TSX Composite index",
    "^BVSP": "IBOVESPA",
    "^MXX": "IPC MEXICO",
    "^IPSA": "S&P/CLX IPSA",
    "^MERV": "MERVAL",
    "^TA125.TA": "TA-125",
    "^CASE30": "EGX 30 Price Return Index",
    "^JN0U.JO": "Top 40 USD Net TRI Index"
}

ticker_name = dft.info.get("longName", "Unknown Ticker")
print(ticker_name)

# # Define the start and end dates for the data
startDate = "1990-01-01"
endDate = "2024-01-01"

timeZone = dft.info.get("timeZoneFullName")
print(timeZone)

df = dft.history(interval="1d", start=startDate, end=endDate)

# Comparing Timestamp
# start_timestamp = int(datetime.datetime.strptime(startDate, "%Y-%m-%d").timestamp())
# print(start_timestamp)
# timestamp1 = 315550800
# timestamp2 = 315507600
# date1 = datetime.datetime.fromtimestamp(timestamp1)
# date2 = datetime.datetime.fromtimestamp(timestamp2)
# print(f"Timestamp 1: {timestamp1} -> Date 1: {date1}")
# print(f"Timestamp 2: {timestamp2} -> Date 2: {date2}")

In [ ]:
# Drop Dividends column
df.drop(columns=['Dividends'], inplace=True)

# Drop Stock Splits column
df.drop(columns=['Stock Splits'], inplace=True)

df

In [ ]:
## Calculate EMA-12 and EMA-26 using Exponential Weighing Average (EWM)
# df['EMA-12'] = df['Close'].ewm(span = 12, adjust = False).mean()
# df['EMA-26'] = df['Close'].ewm(span = 26, adjust = False).mean()

## Calculate MACD 
# df['MACD'] = df['EMA-12'] - df['EMA-26']
df['MACD'] = ta.macd(df['Close'], fast=12, slow=26, signal=9)['MACD_12_26_9']
df['Pct_Change'] = df['Close'].pct_change() * 100

## Calculate RSI using formula
## RSI = 100 – [100 ÷ ( 1 + (Average Gain During Up Periods ÷ Average Loss During Down Periods ))]

delta = df['Close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
rs = gain / loss
df['RSI'] = 100 - (100 / (1 + rs))

delta = df['Close'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)

# Calculate the Exponential Moving Average of gains and losses
avg_gain = gain.ewm(span=14, min_periods=14).mean()
avg_loss = loss.ewm(span=14, min_periods=14).mean()

# Calculate the RS and RSI
rs = avg_gain / avg_loss
df['RSI_EMA'] = 100 - (100 / (1 + rs))

df['RSI_ta'] = ta.rsi(df['Close'], length=14)

df['MA10'] = df.ta.sma(length=10)
df['MA50'] = df.ta.sma(length=50)
df['MA200'] = df.ta.sma(length=200)

## Calculate Boilinger Bands
window = 20
df['MA20'] = df['Close'].rolling(window=window).mean()
df['std_dev'] = df['Close'].rolling(window=window).std()
df['Upper_BB'] = df['MA20'] + (df['std_dev'] * 2)
df['Lower_BB'] = df['MA20'] - (df['std_dev'] * 2)

In [ ]:
df[['Close', 'MACD', 'RSI','RSI_EMA','RSI_ta']]

In [ ]:
# Create target variable: 1 if next day's close is higher than today's, else 0
df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)

In [ ]:
# Fetch VIX index data
vix = yf.Ticker("^VIX")
vix_df = vix.history(interval="1d", start=startDate, end=endDate)

vix_df['Pct_Change'] = vix_df['Close'].pct_change() * 100

# print(vix_df['Pct_Change'].max())


import pytz

# Get the time zone of the stock's exchange
stock_timezone = pytz.timezone(timeZone)

# Convert the time zone of the VIX DataFrame to match the time zone of the stock's exchange
vix_df.index = vix_df.index.tz_convert(stock_timezone)

# Align the timestamps of the VIX data to match those of your DataFrame
vix_df = vix_df.reindex(df.index, method='ffill')

# Merge VIX close prices into the original DataFrame
df['VIX'] = vix_df['Close']



# # df.index = df.index.tz_convert('UTC')
# # # vix_df.index = vix_df.index.tz_convert('UTC')
# # # vix_df.index = vix_df.index.tz_convert('Asia/Bangkok')
# # vix_df.index = vix_df.index.tz_convert('Asia/Beijing')

# # # Shift the timestamps of vix_df by one hour
# # vix_df.index = vix_df.index - pd.Timedelta(hours=1)


# # # Merge VIX close prices into the original DataFrame
# # # df['VIX'] = vix_df['Close']

# # df = df.join(vix_df[['Close']], rsuffix='_VIX')

# # Rename the VIX close column
# df.rename(columns={'Close_VIX': 'VIX'}, inplace=True)

vix_df

In [ ]:
df

In [ ]:
# plt.figure(figsize=(20,6))
# plt.plot(df['Close'], label="Closed Price")
# mx = df['Close'].max()
# plt.plot(vix_df['Close'] * mx / 100, label="VIX Index")
# plt.legend()
# plt.title(f"VIX Index and {ticker_name} Price (Adjusted Index)")
# plt.show()

# Create the plotly figure
fig = go.Figure()

# Add the closed price line
fig.add_trace(go.Scatter(
    x=df.index, 
    y=df['Close'], 
    mode='lines', 
    name='Closed Price',
    hovertemplate='%{x|%Y-%m-%d}<br>Price: %{y}'
))

mx = df['Close'].max()

# Add the VIX index line, Scaled out of Max stock price
fig.add_trace(go.Scatter(
    x=vix_df.index, 
    y=vix_df['Close'] * mx / 100, 
    mode='lines', 
    name='VIX',
    hovertemplate='%{x|%Y-%m-%d}<br>VIX Index: %{customdata:.2f}',
    customdata=vix_df['Close']  # Use the real VIX values as customdata
))

# Update layout
fig.update_layout(
    title=f"VIX Index to {ticker_name} Price (Adjusted Index)",
    xaxis_title="Date",
    yaxis_title="Price",
    legend_title="Legend",
    width=2100,
    height=700
)

# Show the plot
fig.show()

In [ ]:
# plt.figure(figsize=(25,10))
# plt.plot(vix_df['Pct_Change'], label="VIX Index")
# plt.legend()
# plt.title("VIX Index Percentage Change")
# plt.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=vix_df.index, y=vix_df['Close'], mode='lines', name='Percentage Change'))
fig.update_layout(
    title=f"VIX Index Closed Price",
    xaxis_title="Date",
    yaxis_title="Percentage Change",
    legend_title="Legend",
    width=3500,
    height=700
)

fig.show()



In [ ]:
# print("Main DataFrame Index:", df.index)
# print("VIX DataFrame Index:", vix_df.index)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df.index,
    y=df['Close'],
    mode='lines',
    line=dict(color='cyan'),
    name=f"{ticker_name} Price")
)

fig.update_layout(
    title=f"{ticker_name} Price",
    xaxis_title="Date",
    yaxis_title="Price",
    width=1200,
    height=600
)

fig.show()


In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

# Plot the VIX index
fig.add_trace(go.Scatter(
    x=vix_df.index,
    y=vix_df['Close'],
    mode='lines',
    line=dict(color='purple'),
    name='VIX Index')
)

prev_vix = None
gt_vix_date = []
xtr_vix_date = []

# Iterate through the VIX data
for date, vix_value in vix_df['Close'].items():
    if vix_value >= 65 and (prev_vix is None or prev_vix < 65):
        fig.add_trace(go.Scatter(
            x=[date],
            y=[vix_value],
            mode='markers',
            marker=dict(color='green', symbol='x'),
            name=str(date)  # Convert Timestamp to string
        ))
        xtr_vix_date.append([date, vix_value])
    elif vix_value >= 35 and (prev_vix is None or prev_vix < 35):
        fig.add_trace(go.Scatter(
            x=[date],
            y=[vix_value],
            mode='markers',
            marker=dict(color='red', symbol='circle'),
            name=str(date)  # Convert Timestamp to string
        ))
        gt_vix_date.append([date, vix_value])
    prev_vix = vix_value

fig.update_layout(
    title="VIX Index",
    xaxis_title="Date",
    yaxis_title="VIX Value",
    width=4200,
    height=600
)

fig.show()


In [ ]:
# Remove any row with NaN value
df = df.dropna()

# Print out list where the VIX is greater than 30
gt_vix = df[df['VIX'] > 30]
# gt_vix

gt_vix_df = pd.DataFrame(gt_vix_date, columns=['Date', 'VIX'])
gt_vix_df.set_index('Date', inplace=True)

# Convert xtr_vix_date to DataFrame
xtr_vix_df = pd.DataFrame(xtr_vix_date, columns=['Date', 'VIX'])
xtr_vix_df.set_index('Date', inplace=True)

# df['VIX_35'] = df.index.isin(gt_vix_df.index)
# df['VIX_65'] = df.index.isin(xtr_vix_df.index)
df

In [ ]:
cvTest = False


if not cvTest:
    cv_scores = np.array([0.0, 0.0])
    md2_cv_scores = np.array([0.0, 0.0])
    ensemble1_cv_scores = np.array([0.0, 0.0])
    ensemble_cv_scores = np.array([0.0, 0.0])
    
# Prepare the feature set and target variable
X = df[['Pct_Change', 'std_dev', 'MA200', 'MA50', 'MA10', 'MACD', 'RSI_ta', 'VIX']]
# X = df[['Pct_Change', 'std_dev', 'MA200', 'MA50', 'MA10', 'MACD', 'RSI_ta', 'VIX']]
# X = df[['MA200', 'MA50', 'MA10', 'MACD2', 'RSI_ta', 'VIX', 'VIX_35', 'VIX_65']]
# X = df[['SMA200', 'SMA50', 'SMA10', 'MACD', 'RSI', 'VIX', 'VIX_35', 'VIX_65', 'MA20', 'Lower_BB', 'Upper_BB']]
y = df['Target']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Train the RandomForestClassifier
model = RandomForestClassifier(
    n_estimators=100, 
    random_state=42, 
    max_depth=5, 
    min_samples_leaf=1, 
    min_samples_split=10
)

model.fit(X_train, y_train)

print("Train score : ", model.score(X_train,y_train))
print("Test score : ", model.score(X_test,y_test))

from sklearn.model_selection import cross_val_score

if cvTest:
    cv_scores = cross_val_score(model, X, y, cv=5)
    print("Cross-validation scores:", cv_scores)
    print("Mean cross-validation score:", cv_scores.mean())


In [ ]:
y_pred = model.predict(X_test)

# Compute the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Check feature importances
importances = model.feature_importances_
feature_names = X.columns
feature_importance = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
print(feature_importance.sort_values(by='Importance', ascending=False))

# Hyperparameter tuning example using GridSearchCV
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7, 9, 11, 13, 15],
    'min_samples_split': [1, 2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
# grid_search.fit(X_train, y_train)
# print("Best parameters found: ", grid_search.best_params_)
# print("Best cross-validation score: ", grid_search.best_score_)

# # Re-train the model with the best parameters
# best_model = grid_search.best_estimator_
# best_model.fit(X_train, y_train)
# print("Train score with best parameters: ", best_model.score(X_train, y_train))
# print("Test score with best parameters: ", best_model.score(X_test, y_test))



# Output

# Feature  Importance
# 4     RSI    0.207028
# 3    MACD    0.173785
# 5     VIX    0.161541
# 0  SMA200    0.156257
# 2   SMA10    0.150279
# 1   SMA50    0.146876
# 6  VIX_35    0.002741
# 7  VIX_65    0.001494
# Fitting 3 folds for each of 54 candidates, totalling 162 fits
# Best parameters found:  {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100}
# Best cross-validation score:  0.513870182753551
# Train score with best parameters:  0.5857438016528925
# Test score with best parameters:  0.5050147492625369

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

# Initialize XGBClassifier
xgb_model = XGBClassifier(random_state=42)

# Define the grid of hyperparameters to search
param_grid = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
    'max_depth': [3, 5, 7],            # Maximum depth of a tree
    'learning_rate': [0.1, 0.01, 0.001], # Learning rate
    'subsample': [0.8, 0.9],           # Subsample ratio of the training instances
    'colsample_bytree': [0.8, 0.9],    # Subsample ratio of columns when constructing each tree
    'gamma': [0.1, 0.2],               # Minimum loss reduction required to make a further partition
    'reg_alpha': [0.1, 0.2],           # L1 regularization term on weights
    'reg_lambda': [0.1, 0.2]           # L2 regularization term on weights
}

# # Perform grid search with cross-validation
# grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
# grid_search.fit(X_train, y_train)

# # Get the best parameters and score
# best_params = grid_search.best_params_
# best_score = grid_search.best_score_

# # Print the best parameters and score
# print("Best Parameters:", best_params)
# print("Best Score:", best_score)

# # Evaluate the model with the best parameters
# best_xgb_model = grid_search.best_estimator_
# print("XGB Train score:", best_xgb_model.score(X_train, y_train))
# print("XGB Test score:", best_xgb_model.score(X_test, y_test))

# Output

# Best Parameters: {'colsample_bytree': 0.8, 
#     'gamma': 0.1, 
#     'learning_rate': 0.001, 
#     'max_depth': 3, 
#     'n_estimators': 100, 
#     'reg_alpha': 0.1, 
#     'reg_lambda': 0.1, 
#     'subsample': 0.8 
# }
# Best Score: 0.5494392014716607
# XGB Train score: 0.5494391971664699
# XGB Test score: 0.5510324483775811


In [ ]:
pipe = Pipeline([
    ("scaler", MinMaxScaler()), 
    ("classifier", KNeighborsClassifier())
]) 
pipe.fit(X_train, y_train)
class_names = pipe.classes_
class_names
y_pred = pipe.predict(X_test)
acc_train = accuracy_score(y_train, pipe.predict(X_train))
acc_test = accuracy_score(y_test, y_pred)
print(f'Train Accuracy: {acc_train:0.4}, Test Accuracy: {acc_test:0.4}')

if cvTest:
    md2_cv_scores = cross_val_score(model, X, y, cv=5)
    print("Cross-validation scores:", md2_cv_scores)
    print("Mean cross-validation score:", md2_cv_scores.mean())

In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

# Define the models
rf = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=5)
xgb = XGBClassifier(random_state=42, n_estimators=100, max_depth=5, learning_rate=0.00825)

# Create the ensemble
ensemble_model1 = VotingClassifier(estimators=[('rf', rf), ('xgb', xgb)], voting='soft')

# Fit the ensemble
ensemble_model1.fit(X_train, y_train)

# Evaluate the ensemble
print("Ensemble Train score:", ensemble_model1.score(X_train, y_train))
print("Ensemble Test score:", ensemble_model1.score(X_test, y_test))

if cvTest:
    # Cross-validation
    ensemble1_cv_scores = cross_val_score(ensemble_model1, X, y, cv=5)
    print("Ensemble Cross-validation scores:", ensemble1_cv_scores)
    print("Ensemble Mean cross-validation score:", ensemble1_cv_scores.mean())


In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV


# Define hyperparameters grid for Random Forest
rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Define hyperparameters grid for XGBoost
xgb_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001, 0.005, 0.0025, 0.00125],
    'subsample': [0.8, 0.9],
    'colsample_bytree': [0.8, 0.9],
    'gamma': [0.1, 0.2],
    'reg_alpha': [0.1, 0.2],
    'reg_lambda': [0.1, 0.2]
}

# Define models
# rf = RandomForestClassifier(random_state=42)
# xgb = XGBClassifier(random_state=42)

gridSearch = False

rf = RandomForestClassifier(random_state=42, max_depth=5, 
                            min_samples_leaf=1, min_samples_split=5, 
                            n_estimators=91)

xgb = XGBClassifier(random_state=42, colsample_bytree=0.8, 
                    gamma=0.1, learning_rate=0.001, 
                    max_depth=3, n_estimators=100, 
                    reg_alpha=0.1, reg_lambda=0.1, 
                    subsample=0.8)

# Perform GridSearchCV for Random Forest
if gridSearch:
    rf_grid_search = GridSearchCV(estimator=rf, param_grid=rf_param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    rf_grid_search.fit(X_train, y_train)
    best_rf = rf_grid_search.best_estimator_
else: 
    rf.fit(X_train, y_train)

# Perform GridSearchCV for XGBoost
if gridSearch:
    xgb_grid_search = GridSearchCV(estimator=xgb, param_grid=xgb_param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    xgb_grid_search.fit(X_train, y_train)
    best_xgb = xgb_grid_search.best_estimator_
else:
    xgb.fit(X_train, y_train)

# Create the ensemble with best estimators
ensemble_model = VotingClassifier(estimators=[('rf', rf), ('xgb', xgb)], voting='soft')
ensemble_model.fit(X_train, y_train)

# Evaluate the ensemble
print("Ensemble Train score:", ensemble_model.score(X_train, y_train))
print("Ensemble Test score:", ensemble_model.score(X_test, y_test))

# Cross-validation
if cvTest:
    ensemble_cv_scores = cross_val_score(ensemble_model, X, y, cv=5)
    print("Ensemble Cross-validation scores:", ensemble_cv_scores)
    print("Ensemble Mean cross-validation score:", ensemble_cv_scores.mean())


In [ ]:
# print(rf_grid_search.best_params_)
# {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}

# print(xgb_grid_search.best_params_)
# {'colsample_bytree': 0.8, 'gamma': 0.1, 'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 100, 'reg_alpha': 0.1, 'reg_lambda': 0.1, 'subsample': 0.8}

In [ ]:
print("Model 1")
print(f"Train score : {model.score(X_train,y_train):0.5} \nTest score  : {model.score(X_test,y_test):0.5}")
if cvTest: print(f"CV Score    : {cv_scores.mean():0.5}\n")

print("Model 2")
print(f"Train score : {acc_train:0.5} \nTest score  : {acc_test:0.5}")
if cvTest: print(f"CV Score    : {md2_cv_scores.mean():0.5}\n")

print("Model 3")
print(f"Train score : {ensemble_model1.score(X_train, y_train):0.5} \nTest score  : {ensemble_model1.score(X_test, y_test):0.5}")
if cvTest: print(f"CV Score    : {ensemble1_cv_scores.mean():0.5}\n")

print("Model 4")
print(f"Train score : {ensemble_model.score(X_train, y_train):0.5} \nTest score  : {ensemble_model.score(X_test, y_test):0.5}")
if cvTest: print(f"CV Score    : {ensemble_cv_scores.mean():0.5}\n")


In [ ]:
import plotly.graph_objects as go

# Create the figure
fig = go.Figure()

# Add bars for each model with different colors
fig.add_trace(go.Bar(
    x=['Train', 'Test', 'CV'],
    y=[
        model.score(X_train, y_train), 
        model.score(X_test, y_test), 
        cv_scores.mean()
    ],
    name='Model 1',
    marker_color='blue',
    text=[model.score(X_train, y_train), model.score(X_test, y_test), cv_scores.mean()],
    textposition='auto'
))

fig.add_trace(go.Bar(
    x=['Train', 'Test', 'CV'],
    y=[
        acc_train, 
        acc_test, 
        md2_cv_scores.mean()
    ],
    name='Model 2',
    marker_color='orange',
    text=[acc_train, acc_test, md2_cv_scores.mean()],
    textposition='auto'
))

fig.add_trace(go.Bar(
    x=['Train', 'Test', 'CV'],
    y=[
        ensemble_model1.score(X_train, y_train), 
        ensemble_model1.score(X_test, y_test), 
        ensemble1_cv_scores.mean()
    ],
    name='Model 3',
    marker_color='green',
    text=[ensemble_model1.score(X_train, y_train), ensemble_model1.score(X_test, y_test), ensemble1_cv_scores.mean()],
    textposition='auto'
))

fig.add_trace(go.Bar(
    x=['Train', 'Test', 'CV'],
    y=[
        ensemble_model.score(X_train, y_train), 
        ensemble_model.score(X_test, y_test), 
        ensemble_cv_scores.mean()
    ],
    name='Model 4',
    marker_color='red',
    text=[ensemble_model.score(X_train, y_train), ensemble_model.score(X_test, y_test), ensemble_cv_scores.mean()],
    textposition='auto'
))

# Update the layout
fig.update_layout(
    title='Model Scores Comparison',
    xaxis_title='Score',
    yaxis_title='Score out of 1.0',
    barmode='group',
    legend_title='Models',
    xaxis=dict(
        tickmode='array',
        tickvals=['Train', 'Test', 'CV']
    )
)

# Show the plot
fig.show()


In [ ]:
df[df['VIX'] > 40]

In [ ]:
df

In [ ]:
df['Signal'] = 0
# df.loc[X_test.index, 'Signal'] = model.predict(X_test) ## Model 1 Random Forest
# df.loc[X_test.index, 'Signal'] = ensemble_model1.predict(X_test) ## Model 2 Random Forest + XGBoost
df.loc[X_test.index, 'Signal'] = ensemble_model.predict(X_test) ## Model 3 More precise Random Forest + XGBoost 

# Initialize the 'Position' column
df['Position'] = 0



## Iterate through the DataFrame to apply the conditions
holding_position = False

## Condition 1

order_list = []

start_idx = df.index.get_loc(X_test.index[0])

# for i in range(1, len(df)): 
#     if df.loc[df.index[i], 'VIX'] > 30:
#         # print(df.index[i])
#         if df.loc[df.index[i], 'Signal'] == 1 and df.loc[df.index[i], 'VIX'] > 80 and not holding_position:
#             order_list.append({df.index[i],'80'})
#             df.loc[df.index[i], 'Position'] = 1
#             holding_position = True
#         elif df.loc[df.index[i], 'VIX'] > 70 and holding_position and df.loc[df.index[i], 'VIX'] <= 75:
#             df.loc[df.index[i], 'Position'] = 0
#             holding_position = False
#         elif df.loc[df.index[i], 'Signal'] == 1 and df.loc[df.index[i], 'VIX'] > 70 and not holding_position:
#             order_list.append({df.index[i],'70'})
#             df.loc[df.index[i], 'Position'] = 1
#             holding_position = True
#         elif df.loc[df.index[i], 'VIX'] > 65 and holding_position and df.loc[df.index[i], 'VIX'] <= 70:
#             df.loc[df.index[i], 'Position'] = 0
#             holding_position = False
#         elif df.loc[df.index[i], 'Signal'] == 1 and df.loc[df.index[i], 'VIX'] > 55 and not holding_position and df.loc[df.index[i], 'VIX'] <= 65:
#             order_list.append({df.index[i],'55'})
#             df.loc[df.index[i], 'Position'] = 1
#             holding_position = True
#         elif df.loc[df.index[i], 'VIX'] > 45 and holding_position:
#             df.loc[df.index[i], 'Position'] = 0
#             holding_position = False
#         elif df.loc[df.index[i], 'Signal'] == 1 and not holding_position and df.loc[df.index[i], 'VIX'] <= 45:
#         # elif not holding_position and df.loc[df.index[i], 'VIX'] < 50:
#             order_list.append({df.index[i],'30'})
#             df.loc[df.index[i], 'Position'] = 1
#             holding_position = True
#     # elif (df.loc[df.index[i], 'VIX'] < 20 or df.loc[df.index[i], 'Signal'] == 0) and holding_position:
#     elif (df.loc[df.index[i], 'VIX'] < 20 ) and holding_position:
#         df.loc[df.index[i], 'Position'] = 0
#         holding_position = False
        
# for i in range(start_idx, len(df)): 
#     if df.loc[df.index[i], 'VIX'] > 30:
#         print("40")
#         if df.loc[df.index[i], 'VIX'] > 50 and not holding_position:
#             df.loc[df.index[i], 'Position'] = 1
#             holding_position = True
#         elif df.loc[df.index[i], 'VIX'] > 50 and holding_position:
#             df.loc[df.index[i], 'Position'] = 0
#             holding_position = False
#         elif not holding_position and df.loc[df.index[i], 'VIX'] < 40:
#         # elif df.loc[df.index[i], 'Signal'] == 1 and not holding_position and df.loc[df.index[i], 'VIX'] < 50:
#             df.loc[df.index[i], 'Position'] = 1
#             holding_position = True
#     elif (df.loc[df.index[i], 'VIX'] < 20 ) and holding_position:
#         df.loc[df.index[i], 'Position'] = 0
#         holding_position = False
    
    ## or df.loc[df.index[i], 'Signal'] == 0
##  Condition 2

# for i in range(1, len(df)):
#     if df.loc[df.index[i], 'Signal'] == 1 and df.loc[df.index[i], 'VIX'] > 35 and not holding_position:
#         df.loc[df.index[i], 'Position'] = 1
#         holding_position = True
#     elif (df.loc[df.index[i], 'VIX'] < 20 or df.loc[df.index[i], 'Signal'] == 0) and holding_position:
#     # if (df.loc[df.index[i], 'VIX'] < 20 ) and holding_position:
#         # print("Closing position VIX less than 30 at : ", df.index[i])
#         df.loc[df.index[i], 'Position'] = 0
#         # print(df.loc[df.index[i], 'Position'])
#         holding_position = False
#     elif df.loc[df.index[i], 'VIX'] > 45 and holding_position:
#         df.loc[df.index[i], 'Position'] = 0
#         holding_position = False
#     elif df.loc[df.index[i], 'VIX'] > 60 and not holding_position:
#         df.loc[df.index[i], 'Position'] = 1
#         # print("hello", df.index[i])
#         holding_position = True
        
for i in range(1, len(df)):
    if df.loc[df.index[i], 'Signal'] == 1 and not holding_position:
        df.loc[df.index[i], 'Position'] = 1
        holding_position = True
    elif  df.loc[df.index[i], 'Signal'] == 0 and holding_position:
        df.loc[df.index[i], 'Position'] = 0
        holding_position = False


# Forward fill the positions to simulate holding positions
df['Position'] = df['Position'].replace(to_replace=0, method='ffill')

# Calculate strategy returns
# df['Strategy_Returns'] = df['Position'].shift(1) * df['Close'].pct_change()
df_test_period = df.loc[X_test.index[0]:]

# Calculate strategy returns
df_test_period['Strategy_Returns'] = df_test_period['Position'].shift(1) * df_test_period['Close'].pct_change()


# Drop NaN values from returns
# df.dropna(subset=['Strategy_Returns'], inplace=True)
df_test_period.dropna(subset=['Strategy_Returns'], inplace=True)



In [ ]:
print(start_idx, len(df))

In [ ]:
df2 = df[df.index > X_test.index[0]]

In [ ]:
df2

In [ ]:
df2[df2['VIX'] > 40]

In [ ]:
order_list

In [ ]:
len(order_list)

In [ ]:
df_test_period.index

In [ ]:
# import matplotlib.pyplot as plt

# # Calculate cumulative returns
# df['Cumulative_Strategy_Returns'] = (1 + df['Strategy_Returns']).cumprod()

# # Plot cumulative returns
# plt.figure(figsize=(14, 7))
# plt.plot(df.index, df['Cumulative_Strategy_Returns'], label='Strategy Returns')
# plt.title('Cumulative Strategy Returns')
# plt.xlabel('Date')
# plt.ylabel('Cumulative Returns')
# plt.legend()
# plt.grid(True)
# plt.show()

# # Calculate performance metrics
# import numpy as np

# # Calculate annualized return
# annualized_return = df['Strategy_Returns'].mean() * 252

# # Calculate annualized volatility
# annualized_volatility = df['Strategy_Returns'].std() * np.sqrt(252)

# # Calculate Sharpe ratio
# risk_free_rate = 0.01  # Assuming a risk-free rate of 1%
# sharpe_ratio = (annualized_return - risk_free_rate) / annualized_volatility

# # Calculate maximum drawdown
# cumulative_returns = (1 + df['Strategy_Returns']).cumprod()
# running_max = cumulative_returns.cummax()
# drawdown = (cumulative_returns - running_max) / running_max
# max_drawdown = drawdown.min()

# # Print performance metrics
# print(f"Annualized Return: {annualized_return:.2%}")
# print(f"Annualized Volatility: {annualized_volatility:.2%}")
# print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
# print(f"Maximum Drawdown: {max_drawdown:.2%}")


In [ ]:
# Calculate performance metrics
# returns = df['Strategy_Returns']
returns = df_test_period['Strategy_Returns']
cumulative_returns = (1 + returns).cumprod() - 1
perf_stats = {
    'Annual Return': ep.annual_return(returns),
    'Cumulative Returns': ep.cum_returns_final(returns),
    'Annual Volatility': ep.annual_volatility(returns),
    'Sharpe Ratio': ep.sharpe_ratio(returns),
    'Sortino Ratio': ep.sortino_ratio(returns),
    'Max Drawdown': ep.max_drawdown(returns),
    'Calmar Ratio': ep.calmar_ratio(returns)
}

# Print performance metrics
for metric, value in perf_stats.items():
    print(f"{metric}: {value:.5f}")

In [ ]:
# Plot cumulative returns
cumulative_returns = ep.cum_returns(returns, starting_value=1.00)
plt.figure(figsize=(10, 6))
plt.plot(cumulative_returns, label='Strategy Returns')
plt.title('Cumulative Returns')
plt.legend()
plt.show()

In [ ]:
# Calculate rolling Sharpe ratio
def rolling_sharpe_ratio(returns, window=252):
    return returns.rolling(window=window).mean() / returns.rolling(window=window).std()

# Calculate the rolling Sharpe ratio
rolling_sharpe = rolling_sharpe_ratio(returns)

# Plot the rolling Sharpe ratio
plt.figure(figsize=(10, 6))
plt.plot(rolling_sharpe, label='Rolling Sharpe Ratio')
plt.title('Rolling Sharpe Ratio')
plt.xlabel('Date')
plt.ylabel('Sharpe Ratio')
plt.legend()
plt.show()

# Print the mean of the rolling Sharpe ratio
print('Mean of Rolling Sharpe Ratio:', rolling_sharpe.mean())

In [ ]:
# Calculate drawdown
def calculate_drawdown(returns):
    cum_returns = (1 + returns).cumprod()
    peak = cum_returns.cummax()
    drawdown = (cum_returns - peak) / peak
    return drawdown

drawdown = calculate_drawdown(returns)
plt.figure(figsize=(10, 6))
plt.plot(drawdown, label='Drawdown')
plt.title('Drawdown')
plt.legend()
plt.show()

print(drawdown.mean())

print()

In [ ]:
print("Maximum drawdown : ", drawdown.min())

In [ ]:
print(df.loc[df.index[0]])

In [ ]:
# df.to_csv("Backtested/df.csv")
# returns.to_csv("Backtested/returns.csv")